In [12]:
from sagemaker.huggingface.model import HuggingFaceModel
from datasets import Dataset
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import numpy as np
import pandas as pd
import awswrangler as wr
from quality_calculator import compute_bias_metrics_for_model, calculate_overall_auc, get_final_metric
from tqdm import tqdm
tqdm.pandas()
import json

SEED = 1234

import os
import random
import gc
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Today I'm going to use {device.type}")

model_name = 'unitary/unbiased-toxic-roberta'

model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir = '../tmp/AutoModel', return_dict=False)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir = '../tmp/AutoTokenizer')    

Today I'm going to use cpu


In [44]:
model_config = AutoConfig.from_pretrained(model_name)
model_config

RobertaConfig {
  "_name_or_path": "unitary/unbiased-toxic-roberta",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "function_to_apply": "sigmoid",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "toxicity",
    "1": "severe_toxicity",
    "2": "obscene",
    "3": "identity_attack",
    "4": "insult",
    "5": "threat",
    "6": "sexual_explicit",
    "7": "male",
    "8": "female",
    "9": "homosexual_gay_or_lesbian",
    "10": "christian",
    "11": "jewish",
    "12": "muslim",
    "13": "black",
    "14": "white",
    "15": "psychiatric_or_mental_illness"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "black": 13,
    "christian": 10,
    "female": 8,
    "homosexual_gay_or_lesbian": 9,
    "identity_attack": 3,
    "insult": 4,
 

In [45]:
labels = {k: v for k, v in sorted(model_config.label2id.items(), key=lambda item: item[1])}.keys()
labels

dict_keys(['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit', 'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness'])

In [6]:
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()
gc.collect()
torch.cuda.empty_cache()
# del model
# del Trainer
# del tokenizer

In [7]:
with open("../data/godel.txt") as f:
    lines = f.readlines()
godel_test_comments = pd.DataFrame(lines, columns = ['comment_text'])
godel_test_comments['comment_text'] = godel_test_comments['comment_text'].apply(lambda x: x.strip())

In [46]:
sigmoid = torch.nn.Sigmoid()

In [80]:
def predict_toxicity(text):
    inputs = tokenizer(text, max_length=512, padding="max_length",
                       truncation=True, add_special_tokens = True,
                       return_tensors="pt").to(device)
    with torch.no_grad():
        model.to(device)
        outputs = model(**inputs)[0][0]
    probas = sigmoid(outputs).cpu().detach().numpy().astype('str')
    return dict(zip(labels, probas))

In [81]:
predict_toxicity("Ok team what is going on? what the fuck are you doing? we planed to go to bar but instead of that we going to theatre")

{'toxicity': '0.93758357',
 'severe_toxicity': '0.000119789875',
 'obscene': '0.8746471',
 'identity_attack': '0.0028658106',
 'insult': '0.32462212',
 'threat': '0.006009663',
 'sexual_explicit': '0.004311522',
 'male': '0.0029190886',
 'female': '0.0006424607',
 'homosexual_gay_or_lesbian': '7.274166e-05',
 'christian': '0.00094889704',
 'jewish': '0.0004826698',
 'muslim': '0.0004454623',
 'black': '0.00011403677',
 'white': '0.0015914162',
 'psychiatric_or_mental_illness': '0.00037234882'}

In [84]:
%%time
godel_test_comments['results'] = godel_test_comments['comment_text'].apply(predict_toxicity)

CPU times: user 38.1 s, sys: 236 ms, total: 38.4 s
Wall time: 20.4 s


In [85]:
godel_test_comments

,comment_text,results
0,"Hi team, August 15th is a state holiday in Poland, so do I understand correctly that we will receive our salaries on Friday, 12th?","{'toxicity': '0.00039274635', 'severe_toxicity': '1.0830036e-06', 'obscene': '3.3286047e-05', 'identity_attack': '2.689828e-05', 'insult': '0.00012478633', 'threat': '2.4396324e-05', 'sexual_explicit': '1.9794405e-05', 'male': '0.00023151858', 'female': '0.00011280523', 'homosexual_gay_or_lesbian': '2.905921e-05', 'christian': '6.1712955e-05', 'jewish': '2.596627e-05', 'muslim': '5.6061337e-05', 'black': '3.3424192e-05', 'white': '7.089801e-05', 'psychiatric_or_mental_illness': '9.86178e-05'}"
1,"Or on Tuesday, 16th","{'toxicity': '0.00041385883', 'severe_toxicity': '1.0956902e-06', 'obscene': '3.2324922e-05', 'identity_attack': '2.476689e-05', 'insult': '0.00012722392', 'threat': '3.2140157e-05', 'sexual_explicit': '1.9178126e-05', 'male': '0.00017618484', 'female': '0.00011390386', 'homosexual_gay_or_lesbian': '2.8550185e-05', 'christian': '8.975274e-05', 'jewish': '3.041413e-05', 'muslim': '6.2180756e-05', 'black': '2.6844824e-05', 'white': '4.7020094e-05', 'psychiatric_or_mental_illness': '9.0528665e-..."
2,"according to contract invoices have to be payed within 15 day, no exceptions for public holidays were described. paying on Tuesday, 16th seems to be a breach of contract. Viktoryia Charnianina please correct me if I'm wrong","{'toxicity': '0.0003846786', 'severe_toxicity': '1.1514799e-06', 'obscene': '3.2395594e-05', 'identity_attack': '2.5553521e-05', 'insult': '0.00011509054', 'threat': '2.7524691e-05', 'sexual_explicit': '2.0613277e-05', 'male': '0.00021106348', 'female': '0.00011042611', 'homosexual_gay_or_lesbian': '2.9396611e-05', 'christian': '8.204769e-05', 'jewish': '2.6920068e-05', 'muslim': '5.6744262e-05', 'black': '3.81562e-05', 'white': '5.767717e-05', 'psychiatric_or_mental_illness': '0.00010327914'}"
3,"Hi folks, any news maybe on this vital topic: 12 or 16?","{'toxicity': '0.00039160624', 'severe_toxicity': '1.3612955e-06', 'obscene': '3.4963654e-05', 'identity_attack': '2.8169505e-05', 'insult': '0.000110241046', 'threat': '3.358209e-05', 'sexual_explicit': '2.2021615e-05', 'male': '0.00021865466', 'female': '0.0001411924', 'homosexual_gay_or_lesbian': '3.6011952e-05', 'christian': '6.689095e-05', 'jewish': '3.7520553e-05', 'muslim': '7.4861935e-05', 'black': '3.6614078e-05', 'white': '6.640044e-05', 'psychiatric_or_mental_illness': '0.000122852..."
4,Viktoryia Charnianina can we get any response?,"{'toxicity': '0.00057641533', 'severe_toxicity': '1.3175003e-06', 'obscene': '4.8019367e-05', 'identity_attack': '2.7453018e-05', 'insult': '0.00020491739', 'threat': '3.708705e-05', 'sexual_explicit': '2.8301118e-05', 'male': '0.00021701159', 'female': '7.7622375e-05', 'homosexual_gay_or_lesbian': '2.635249e-05', 'christian': '8.28833e-05', 'jewish': '1.9627296e-05', 'muslim': '4.3570686e-05', 'black': '4.2056592e-05', 'white': '5.824137e-05', 'psychiatric_or_mental_illness': '0.00016685232'}"
5,"Olesia Buldakova I'm no Viktoryia, but you are wrong Who says you get paid on 16th? Nobody said a word about 16th. And why random person illegitimate post makes you threaten with breach of contract, be nervous etc. I say, I you see a post by Godel administration stating this and that, you can, but this is not the case. Especially tagging busy person responsible for our payments, and she, instead of doing her job, has to now read these void compaints and delay the payments Tag post author in...","{'toxicity': '0.00041481448', 'severe_toxicity': '1.07972e-06', 'obscene': '3.2743523e-05', 'identity_attack': '2.2019538e-05', 'insult': '0.0001483191', 'threat': '2.4928617e-05', 'sexual_explicit': '1.9331452e-05', 'male': '0.00024075738', 'female': '0.00010993462', 'homosexual_gay_or_lesbian': '2.4932873e-05', 'christian': '6.6498535e-05', 'jewish': '2.3337465e-05', 'muslim': '5.1741834e-05', 'black': '3.430184e-05', 'white': '5.9405924e-05', 'psychiatric_or_mental

In [17]:
BUCKET_NAME = 'sagemaker-godeltech'
TEST_PATH = f"s3://{BUCKET_NAME}/data/test/test.csv"
test = wr.s3.read_csv([TEST_PATH])

In [18]:
%%time
results = test['comment_text'].progress_apply(predict_toxicity)
# results = np.vectorize(predict_toxicity)(test['comment_text'])

100%|██████████| 194641/194641 [2:02:56<00:00, 26.39it/s]  

CPU times: user 1h 28min 4s, sys: 35min 18s, total: 2h 3min 23s
Wall time: 2h 2min 56s


In [19]:
predictions = np.where(pd.json_normalize(results)['toxicity'] >= 0.5, 1, 0)

In [20]:
oof_name = 'predicted_target'
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
test[oof_name] = predictions
#evaluation
bias_metrics_df = compute_bias_metrics_for_model(test, identity_columns, oof_name, 'toxicity')
display(bias_metrics_df)
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test, oof_name))
print(f"FINAL SCORE FOR TOXIC-BERT IS {FINAL_SCORE}")  

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
5,muslim,2040,0.612840,0.706424,0.612986
7,white,2452,0.620279,0.696593,0.631096
6,black,1519,0.626930,0.695862,0.637012
3,christian,4226,0.627449,0.709378,0.623848
2,homosexual_gay_or_lesbian,1065,0.639722,0.682649,0.661615
4,jewish,835,0.642204,0.696075,0.650283
0,male,4386,0.677810,0.698222,0.684248
1,female,5155,0.679279,0.696321,0.687569
8,psychiatric_or_mental_illness,511,0.708910,0.690462,0.721993


FINAL SCORE FOR TOXIC-BERT IS 0.6741386142708594


In [33]:
from utils import save_to_s3, get_from_s3
from datetime import datetime

TODAY = datetime.today().strftime("%Y%m%d")
BUCKET_NAME = 'sagemaker-godeltech'
MODEL_PATH = "transformers/results"


np.savetxt(f"../tmp/pretrained_predictions_{TODAY}.csv", predictions, delimiter=",")
save_to_s3(BUCKET_NAME, f"../tmp/pretrained_predictions_{TODAY}.csv", f"{MODEL_PATH}/pretrained_predictions_{TODAY}.csv")